In [1]:
from qdrant_client import QdrantClient
from pandas import DataFrame
from PIL import Image
import os 
import pandas as pd

qdrant_client = QdrantClient(
    url="https://b088d693-20f0-4200-95c0-5ba21d0ace8c.us-east4-0.gcp.cloud.qdrant.io:6333", 
    api_key="uT8Jv8SlesDJLW0BEMS3t0JwZOjz_PthtzCVVaedMXLkKv2oI24MIQ",
)

In [15]:
import os
import pandas as pd
from PIL import Image
from io import BytesIO
import math
import base64

class ImageDataset:
    def __init__(self, base_path, num_samples_per_class=50):
        self.base_path = base_path
        self.num_samples_per_class = num_samples_per_class
        self.df = pd.DataFrame()
        self._load_data()

    def _load_data(self):
        for class_id, class_path in enumerate(os.listdir(self.base_path)):
            class_file = os.listdir(f'{self.base_path}/{class_path}')
            samples_in_class = len(class_file)
            samples_in_class = min(samples_in_class, self.num_samples_per_class)
            
            train_samples_files = class_file[:samples_in_class]
            train_samples_files = list(map(lambda x: f'{self.base_path}/{class_path}/{x}', train_samples_files))
            
            payload = pd.DataFrame({
                'image_path': train_samples_files,
                'type': class_path
            })
            self.df = pd.concat([self.df, payload])

    def resize_image(self, image, target_width=256):
        image_aspect_ratio = image.size[0] / image.size[1]
        resized_img = image.resize(
            [target_width, math.floor(target_width * image_aspect_ratio)]
        )
        return resized_img
    
    def convert_image_to_base64(self, image):
        buffered = BytesIO()
        image.save(buffered, format="JPEG")
        return base64.b64encode(buffered.getvalue()).decode("utf-8")

    def __getitem__(self, idx):
        if idx >= len(self.df):
            raise IndexError("Index out of range")
        
        row = self.df.iloc[idx]
        image = Image.open(row['image_path'])
        image = self.resize_image(image)
        encoded_image = self.convert_image_to_base64(image)
        
        return image, row['type'], encoded_image

    def __len__(self):
        return len(self.df)

# Example usage:
TRAIN_BASE_PATH = '../afhq/train'
NUM_SAMPLE_FOR_CLASS = 50

data = ImageDataset(TRAIN_BASE_PATH, NUM_SAMPLE_FOR_CLASS)

In [31]:
df_origin = pd.read_excel('../Data/instagram_posts.xlsx')
df_origin

In [ ]:
# TRAIN_BASE_PATH = '../afhq/train'
# NUM_SAMPLE_FOR_CLASS = 50
# 
# df = pd.DataFrame()
# for class_id, class_path in enumerate(os.listdir(TRAIN_BASE_PATH)):
#     class_file = os.listdir(f'{TRAIN_BASE_PATH}/{class_path}')
#     samples_in_class = len(class_file)
#     samples_in_class = min(samples_in_class, NUM_SAMPLE_FOR_CLASS)
#     
#     train_samples_files = class_file[:samples_in_class]
#     train_samples_files = list(map(lambda x: f'{TRAIN_BASE_PATH}/{class_path}/{x}', train_samples_files))
#     train_samples_images = list(map(lambda x: Image.open(x), train_samples_files))
#     
#     payload = DataFrame.from_records({
#         'image_path': train_samples_files,
#     })
#     payload['type'] = class_path
#     df = pd.concat([df, payload])
# df

In [9]:
import os

train_files = os.listdir('../afhq/train/cat')

train_samples_files = train_files[:200]
train_samples_files = list(map(lambda x: f'../afhq/train/cat/{x}', train_samples_files))
train_samples_images = list(map(lambda x: Image.open(x), train_samples_files))

In [11]:
payload = DataFrame.from_records({
        'image_path': train_samples_files,
    })
payload['type'] = 'cat'
payload

In [12]:
from io import BytesIO
import math
import base64

target_width = 256
def resize_image(image_path):
    pil_img = Image.open(image_path)
    image_aspect_ratio = pil_img.size[0] / pil_img.size[1]
    resized_img = pil_img.resize(
        [target_width, math.floor(target_width * image_aspect_ratio)]
    )
    return resized_img

def convert_image_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


resized_images = list(map( lambda x: resize_image(x), train_samples_files))
base64_str = list(map( lambda x: convert_image_to_base64(x), resized_images))
payload['image'] = base64_str

In [13]:
payload

In [16]:
resized_images = []
for i in range(len(data)):
    resized_image, _, _ = data[i]
    resized_images.append(resized_image)
resized_images

In [17]:
from transformers import AutoImageProcessor, ResNetForImageClassification

processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

inputs = processor(images=resized_images, return_tensors="pt")
outputs = model(**inputs)

In [18]:
del resized_images

In [19]:
embeddings = outputs.logits
embeddings_length = embeddings.shape[1]
embeddings, embeddings_length

In [20]:
from qdrant_client.models import VectorParams, Distance, OptimizersConfigDiff

collection_name = 'animals_pictures'
collection = qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=embeddings_length,
        distance=Distance.COSINE
    ),
    optimizers_config=OptimizersConfigDiff(indexing_threshold=0),
)
collection

In [24]:
# payload_2 = payload.copy()
# payload_dicts = payload_2.to_dict(orient='records')
# payload_dicts

In [22]:
# from qdrant_client.grpc import PointStruct
# from qdrant_client.grpc.points_pb2 import PointId
# 
# points = []
# for idx, payload_dict in enumerate(payload):
#     point_id = PointId(num=idx)  # Create a PointId object
#     vector = embeddings[idx]
#     payload = payload_dict
# 
#     points.append(PointStruct(id=point_id, vector=vector, payload=payload))
# 
# print(points)  # Now, you should see correctly structured PointStruct instances


In [27]:
data_dict = {}
for i in range(len(data)):
    image, image_type, encoded_image = data[i]
    data_dict[i] = {'type': image_type, 'image': encoded_image}

In [28]:
data_dict

In [29]:
from qdrant_client import models

records = [
    models.PointStruct(
        id=idx,
        payload=data_dict[idx],
        vector=embeddings[idx].tolist()
    )
    for idx, _ in enumerate(data_dict)
]
records

In [30]:
qdrant_client.upsert(
    collection_name=collection_name,
    points=records
)

In [ ]:
# qdrant_client.upload_points(
#     collection_name=collection_name,
#     points=records
# )